In [32]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [33]:
import nltk

In [34]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [35]:
import pandas as pd

In [36]:
fake = pd.read_csv("Fake.csv", engine='python', error_bad_lines=False)
genuine = pd.read_csv("True.csv", engine='python', error_bad_lines=False)

Skipping line 3091: unexpected end of data
Skipping line 3309: unexpected end of data


In [37]:
display(fake.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3089 entries, 0 to 3088
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    3089 non-null   object
 1   text     3089 non-null   object
 2   subject  3089 non-null   object
 3   date     3089 non-null   object
dtypes: object(4)
memory usage: 96.7+ KB


None

In [38]:
display(genuine.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3307 entries, 0 to 3306
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    3307 non-null   object
 1   text     3307 non-null   object
 2   subject  3307 non-null   object
 3   date     3307 non-null   object
dtypes: object(4)
memory usage: 103.5+ KB


None

In [39]:
fake['target']=0
genuine['target']=1

In [40]:
data = pd.concat([fake, genuine], axis=0)

In [41]:
data=data.reset_index(drop=True)

In [42]:
data=data.drop(['subject', 'date', 'title'], axis=1)

In [43]:
print(data.columns)

Index(['text', 'target'], dtype='object')


In [44]:
from nltk.tokenize import word_tokenize

In [45]:
data['text']=data['text'].apply(word_tokenize)

In [46]:
from nltk.stem.snowball import SnowballStemmer
porter = SnowballStemmer("english")

In [47]:
def stem_it(text):
    return [porter.stem(word) for word in text]

In [48]:
data['text']=data['text'].apply(stem_it)

In [49]:
data['text']=data['text'].apply(' '.join)

In [50]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test=train_test_split(data['text'], data['target'], test_size=0.25)
display(X_train.head())
print('\n')
display(Y_train.head())

1159    the world was stun when donald trump pull the ...
3452    moscow ( reuter ) - the u.s. embassi to russia...
3739    washington ( reuter ) - the republican and dem...
6245    washington ( reuter ) - the trump administr pl...
2523    donald trump may want to rethink his wall idea...
Name: text, dtype: object

1159    0
3452    1
3739    1
6245    1
2523    0
Name: target, dtype: int64

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer
my_tfidf = TfidfVectorizer(max_df=0.75)

tfidf_train = my_tfidf.fit_transform(X_train)
tfidf_test = my_tfidf.transform(X_test)

In [52]:
print(tfidf_train)

  (0, 30751)	0.023545720778660354
  (0, 14690)	0.021931387480068554
  (0, 11410)	0.02392384599290815
  (0, 19835)	0.039288462451353395
  (0, 7433)	0.04891219374723972
  (0, 2909)	0.036134640585537295
  (0, 13093)	0.03940039487739104
  (0, 18173)	0.024598987063682284
  (0, 14060)	0.016528178990926793
  (0, 23870)	0.0612987215024253
  (0, 12897)	0.03521716549615899
  (0, 13407)	0.021172811497089654
  (0, 18082)	0.07760124092470133
  (0, 4179)	0.05540043799161056
  (0, 19549)	0.021662509764594495
  (0, 23691)	0.04144855058841352
  (0, 31661)	0.06091467945222244
  (0, 14243)	0.03764145553030497
  (0, 16179)	0.023685380629027922
  (0, 14327)	0.03822111307052587
  (0, 22540)	0.033675912303733284
  (0, 13498)	0.06943998891211177
  (0, 29222)	0.05606621620077409
  (0, 11782)	0.06635908087693694
  (0, 30647)	0.03859492948980336
  :	:
  (4796, 27733)	0.049814292700224
  (4796, 22006)	0.022539670625999715
  (4796, 20120)	0.029560158648513142
  (4796, 26073)	0.07322835432964755
  (4796, 29153)	0.0

In [53]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [54]:
model_1 = LogisticRegression(max_iter=800)
model_1.fit(tfidf_train, Y_train)
pred_1 = model_1.predict(tfidf_test)
cr1 = accuracy_score(Y_test, pred_1)
print(cr1*100)

98.49906191369607


In [55]:
from sklearn.linear_model import PassiveAggressiveClassifier

model = PassiveAggressiveClassifier(max_iter=80)
model.fit(tfidf_train, Y_train)

PassiveAggressiveClassifier(max_iter=80)

In [56]:
Y_pred = model.predict(tfidf_test)
accscore = accuracy_score(Y_test, Y_pred)
print('The accuracy of prediction is ', accscore*100)

The accuracy of prediction is  99.56222639149468
